## The Apprentice UK Outcomes

In [3]:
import sqlite3
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

The data we want to see is in the "Performance Chart" section and the "Episodes" summaries. These are tables are offset indexed 1 and 2 respectively.

I'm going to extract the data I need iteratively and load to a db - this is in part cause I started this project on the long train home, relying on a meagre pay-as-you-go limit, and in part as I don't any database management today and want to get some practice in!

### Episode Summaries data

Staring with the Episodes table, there are 2 row classes that we care about: `expand-child` and `vevent module-episode-list-row`. These contain an episode summary and episode key data respectively, which we will need to join together.

In [11]:
# For dropping table
with sqlite3.connect("database.db") as conn:
    conn.execute("DROP TABLE IF EXISTS episode_summaries")
    tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
    print("Tables:", tables)

Tables: []


In [12]:
## Run the first time
with sqlite3.connect("database.db") as conn:
    conn.execute("CREATE TABLE episode_summaries (episode_id INTEGER, episode_num INTEGER, task_title TEXT, task_description TEXT, series_number INTEGER)")
    tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
    print("Tables:", tables)

Tables: [('episode_summaries',)]


In [13]:
def get_and_store_page_content(series_num: int, SNMAX=19):
    """
    Fetches the content of a Wikipedia page for a given series number and stores it locally.
    SNMAX (int, optional): The maximum series number, currently 19 (20 ongoing at time of writing).
    """
    assert series_num > 0 and series_num <= SNMAX and isinstance(series_num, int), "series_num must be a positive integer below 20"
    url = f"https://en.m.wikipedia.org/wiki/The_Apprentice_(British_TV_series)_series_{series_num}"
    page = requests.get(url)
    page_content = page.content
    with open(f"pages/series_{series_num}.html", "wb") as f:
        f.write(page_content)
    return page_content

def get_html_tables(series_num: int, bypass_page_check=False):
    assert series_num > 0 and isinstance(series_num, int), "series_num must be a positive integer"
    try:
        with open(f"pages/series_{series_num}.html", "r") as f:
            page_content = f.read()
    except FileNotFoundError:
        if bypass_page_check or input("Page not found. Download page? (y/n)").lower() == "y":
            page_content = get_and_store_page_content(series_num)
        else:
            return None
    soup = BeautifulSoup(page_content, 'html.parser')
    return soup.find_all('table', class_='wikitable')

def add_episode_summaries_row(series_num: int, bypass_page_check=False, db_path="database.db"):
    """
    Fetches and parses the episode summaries for a given series number, and stores them in the database.
    Table index 2 is the episode summaries table.
    """
    html_table = get_html_tables(series_num, bypass_page_check=bypass_page_check)[2]
    parsed_rows = parse_html_table(html_table)
    if parsed_rows is None:
        return None
    
    ## Add the series number to the parsed rows
    parsed_rows = [(row[0], row[1], row[2], row[3], series_num) for row in parsed_rows]

    ## We need to use a SQL parameter method to insert data with odd characters like '
    sql_template = "INSERT INTO episode_summaries(episode_id, episode_num, task_title, task_description, series_number) VALUES(?, ?, ?, ?, ?)"
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    cur.executemany(sql_template, parsed_rows)
    conn.commit()
    conn.close()
    
    return parsed_rows

def parse_episode_header(html_row):
    """
    Parses the header row of an episode table.
    """
    header = html_row.find_all('th')
    assert len(header) == 1, "Header row must have exactly header one cell"
    episode_id = header[0].text
    cells = html_row.find_all('td')
    episode_num = cells[0].text
    task = cells[1].text
    return episode_id, episode_num, task

def parse_episode_summary(html_row):
    return html_row.text.replace("\n", "")

def parse_html_table(html_table):
    """
    Parses an HTML table of episode summaries into a list of episodal rows represented by tuples.
    Each row is a header html_row followed by a summary html_row, which need to be collected into one
    tuple.
    """
    html_rows = html_table.find_all('tr')
    end_rows = []
    current_row = [0,0,0,0]
    for html_row in html_rows:
        if html_row.get('class') == ['vevent', 'module-episode-list-row']:
            parsed_table_row = parse_episode_header(html_row)

            ## We ignore the special (retrospective) episodes, as they give us
            ## no new information about the competition, and their formats are
            ## different. Special episode numbers aren't entirely numeric
            if not parsed_table_row[1].isnumeric():
                continue
            current_row[0], current_row[1], current_row[2] = parsed_table_row

            # The episode title format needs to be parsed
            current_row[2] = re.search(r"\"(.+)\"", current_row[2]).group(1)
        elif html_row.get('class') == ['expand-child']:
            if current_row[0] == 0:
                continue
            current_row[3] = parse_episode_summary(html_row)
            end_rows.append(current_row)
            current_row = [0,0,0,0]
    return end_rows

In [14]:
## Run the first time
for i in range(1, 20):
    add_episode_summaries_row(i, bypass_page_check=True);
    

In [16]:
df_es = pd.read_sql("SELECT * FROM episode_summaries", sqlite3.connect("database.db"), index_col="episode_id")
df_es

,episode_num,task_title,task_description,series_number
episode_id,,,,
1,1,Wilting Blooms,Sir Alan Sugar begins a search for an apprenti...,1
2,2,Child's Play,Teams put their creative skills to the test wh...,1
3,3,Shop Till You Drop,"On their next task, Sir Alan assigns both team...",1
4,4,The Harrods Sales,Given a concession stand to run within Harrods...,1
5,5,Art with a Capital F,Both teams find themselves each representing a...,1
...,...,...,...,...
250,2,Virtual Pop Star,The second task has the teams assigned to crea...,19
251,3,Discount Buying,The candidates are sent to Stratford-upon-Avon...,19
252,4,Crops to Cash,Both teams are challenged to create products u...,19


In [ ]:
# df_es.to_csv("episode_summaries.csv")

### Performance Chart data

In [7]:
with sqlite3.connect("database.db") as conn:
    conn.execute("DROP TABLE IF EXISTS performance_chart")
    tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
    print("Tables:", tables)

Tables: [('episode_summaries',)]


The chart schema is as follows:
- `Candidate` - the name of the candidate (Primary Key) TEXT
- `Week n result` - the performance of the candidate in week n (where n is the week number) TEXT. This will be one of
    - `PM` - Project Manager and fired
    - `FIRED` - Fired but not PM
    - `BR` - Brought back to the boardroom but not fired
    - `LOSE` - Lost task as PM but not fired
    - `LOSS` - Lost task but not fired or brought back
    - `IN` - In the winning team
    - `WIN` - Won task as PM
    - `RUNNER UP` - Runner up in the final
    - `WINNER` - Winner of the series




In [18]:
SCHEMA = [f'week_{n}_result TEXT' for n in range(1, 12)]

with sqlite3.connect("database.db") as conn:
    conn.execute(f"CREATE TABLE performance_chart (candidate TEXT, {', '.join(SCHEMA)})")
    tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
    print("Tables:", tables)

Tables: [('episode_summaries',), ('performance_chart',)]


Full candidate names are in the 0 index table.